In [267]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC
from sklearn import random_projection

In [2]:
df = pd.read_csv('/media/raian/Projects/Datasets/UCI-HeartDisease/processed.cleveland_test.csv', header=None)
df.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
df.shape

(303, 14)

In [3]:
# converting to binary class
# replace class 1,2,3,4 by 1

df['target'] = df['target'].mask(df['target']>0, 1)

# count of tuples in class 1 after replacement

df['target'][df['target'] == 1].value_counts().sum()

139

In [4]:
# Replace '?' by np.nan

df = df.replace('?',np.nan)

# Find missing values np.nan

df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
target      0
dtype: int64

In [5]:
# Impute np.nan with Most Frequent values

df_imp = df.copy(deep=True)
df_imp
imp = SimpleImputer(strategy='most_frequent')
df_imp = pd.DataFrame(imp.fit_transform(df_imp),columns=df_imp.columns)
df_imp
df_imp.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [6]:
df.ca = df.ca.astype('float64')
df.thal = df.thal.astype('float64')
df.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target        int64
dtype: object

In [7]:
df_imp = df_imp.astype('float64')
df_imp.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target      float64
dtype: object

In [8]:
X = df_imp.drop(['target'],axis=1)

y = df['target']

In [9]:
sc = StandardScaler()
X_norm = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

# Apply a Classification Algorithm

In [10]:
# define the model

def classification(X_data, cls_name, scoring_method):
    model = cls_name
    
# evaluate model

    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X_data, y, scoring=scoring_method, cv=cv)
#     print(n_scores)
#     print(n_scores.shape)
# report performance
#     print('Accuracy: mean = %f ' % (n_scores.mean()))
    return n_scores.mean()

In [176]:
scoring = ['accuracy','precision','recall','f1','roc_auc']

In [191]:
# params = { 'C':[0.01,0.1,1,10,50,100], 'n':[3,4,5,6,7,8,9,10] }
N_FEATURES_OPTIONS = [3,4,5,6,7,8,9,10]
C_OPTIONS = [0.01,0.1,1,10,50,100]

In [186]:
# pipe = Pipeline([
#     # the reduce_dim stage is populated by the param_grid
#     ('reduce_dim', 'passthrough'),
#     ('classify', LogisticRegression())
# ])

In [195]:
# param_grid = [
#     {
#         'reduce_dim': [RFE(estimator = LogisticRegression())],
#         'reduce_dim__n_features_to_select': N_FEATURES_OPTIONS,
#         'classify__C': C_OPTIONS
#     },
#     {
#         'reduce_dim': [LDA(solver='eigen', shrinkage='auto')],
#         'reduce_dim__n_components': N_FEATURES_OPTIONS,
#         'classify__C': C_OPTIONS
#     },
# ]

In [181]:
# lr = LogisticRegression()

In [203]:
# gridcv = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid,scoring=scoring,refit=False)
# grid = gridcv.fit(X, y)

In [204]:
# X_t = grid.transform(X) #problem

### L1 on Actual Data

In [243]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(303, 7)

In [249]:
Scores_L1 = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_L1['#Features'] = [7]
Scores_L1['Classification_Method'] = 'LR'
Scores_L1

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR,7,NaN,NaN,NaN,NaN,NaN


In [250]:
Scores_L1['accuracy'] = classification(X_data=X_new, cls_name=LogisticRegression(), scoring_method='accuracy')
Scores_L1['precision'] = classification(X_data=X_new, cls_name=LogisticRegression(), scoring_method='precision')
Scores_L1['recall'] = classification(X_data=X_new, cls_name=LogisticRegression(), scoring_method='recall')
Scores_L1['f1'] = classification(X_data=X_new, cls_name=LogisticRegression(), scoring_method='f1')
Scores_L1['roc_auc'] = classification(X_data=X_new, cls_name=LogisticRegression(), scoring_method='roc_auc')

Scores_L1

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR,7,0.830645,0.841255,0.781502,0.807613,0.88624


### L1 on Normalized Data

In [254]:
lsvc1 = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_norm, y)
model1 = SelectFromModel(lsvc1, prefit=True)
X_new1 = model1.transform(X_norm)
X_new1.shape

(303, 6)

In [265]:
Scores_L1_norm = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_L1_norm['#Features'] = [6]
Scores_L1_norm['Classification_Method'] = 'LR (C=0.1)'
Scores_L1_norm

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.1),6,NaN,NaN,NaN,NaN,NaN


In [266]:
Scores_L1_norm['accuracy'] = classification(X_data=X_new1, cls_name=LogisticRegression(C=0.1), scoring_method='accuracy')
Scores_L1_norm['precision'] = classification(X_data=X_new1, cls_name=LogisticRegression(C=0.1), scoring_method='precision')
Scores_L1_norm['recall'] = classification(X_data=X_new1, cls_name=LogisticRegression(C=0.1), scoring_method='recall')
Scores_L1_norm['f1'] = classification(X_data=X_new1, cls_name=LogisticRegression(C=0.1), scoring_method='f1')
Scores_L1_norm['roc_auc'] = classification(X_data=X_new1, cls_name=LogisticRegression(C=0.1), scoring_method='roc_auc')

Scores_L1_norm

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.1),6,0.844014,0.875335,0.776923,0.819101,0.89675


# Feature Subset with Gaussian random projection

### Actual Dataset

In [283]:
transformer = random_projection.GaussianRandomProjection(n_components=9,random_state=1)
X_G = transformer.fit_transform(X)
X_G.shape

(303, 9)

In [292]:
Scores_G = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_G['#Features'] = [9]
Scores_G['Classification_Method'] = 'LR (C=10)'
Scores_G

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=10),9,NaN,NaN,NaN,NaN,NaN


In [293]:
Scores_G['accuracy'] = classification(X_data=X_G, cls_name=LogisticRegression(C=10), scoring_method='accuracy')
Scores_G['precision'] = classification(X_data=X_G, cls_name=LogisticRegression(C=10), scoring_method='precision')
Scores_G['recall'] = classification(X_data=X_G, cls_name=LogisticRegression(C=10), scoring_method='recall')
Scores_G['f1'] = classification(X_data=X_G, cls_name=LogisticRegression(C=10), scoring_method='f1')
Scores_G['roc_auc'] = classification(X_data=X_G, cls_name=LogisticRegression(C=10), scoring_method='roc_auc')

Scores_G

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=10),9,0.810824,0.800329,0.788645,0.791035,0.882212


### Normalized Dataset (doesn't perform well on normalized data)

In [430]:
transformer1 = random_projection.GaussianRandomProjection(n_components=9,random_state=1)
X_G_norm = transformer1.fit_transform(X_norm)
X_G_norm.shape

(303, 9)

In [431]:
classification(X_data=X_G_norm, cls_name=LogisticRegression(C=10), scoring_method='accuracy')

0.7813978494623656

# Sparse random projection

## n=5, LR(C=1)

In [432]:
SRP1 = random_projection.SparseRandomProjection(n_components=5,random_state=1)
X_SRP1 = SRP1.fit_transform(X_norm)
X_SRP1.shape
# n=9, C=0.01
# n=8, C=0.1
#n=5, C=1

(303, 5)

In [434]:
Scores_SRP1 = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_SRP1['#Features'] = [5]
Scores_SRP1['Classification_Method'] = 'LR'
Scores_SRP1

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR,5,NaN,NaN,NaN,NaN,NaN


In [435]:
Scores_SRP1['accuracy'] = classification(X_data=X_SRP1, cls_name=LogisticRegression(C=1), scoring_method='accuracy')
Scores_SRP1['precision'] = classification(X_data=X_SRP1, cls_name=LogisticRegression(C=1), scoring_method='precision')
Scores_SRP1['recall'] = classification(X_data=X_SRP1, cls_name=LogisticRegression(C=1), scoring_method='recall')
Scores_SRP1['f1'] = classification(X_data=X_SRP1, cls_name=LogisticRegression(C=1), scoring_method='f1')
Scores_SRP1['roc_auc'] = classification(X_data=X_SRP1, cls_name=LogisticRegression(C=1), scoring_method='roc_auc')

Scores_SRP1

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR,5,0.803154,0.809325,0.755678,0.77708,0.877408


## n=8, LR(C=0.1)

In [436]:
SRP2 = random_projection.SparseRandomProjection(n_components=8,random_state=1)
X_SRP2 = SRP2.fit_transform(X_norm)
X_SRP2.shape

(303, 8)

In [437]:
Scores_SRP2 = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_SRP2['#Features'] = [8]
Scores_SRP2['Classification_Method'] = 'LR (C=0.1)'
Scores_SRP2

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.1),8,NaN,NaN,NaN,NaN,NaN


In [438]:
Scores_SRP2['accuracy'] = classification(X_data=X_SRP2, cls_name=LogisticRegression(C=0.1), scoring_method='accuracy')
Scores_SRP2['precision'] = classification(X_data=X_SRP2, cls_name=LogisticRegression(C=0.1), scoring_method='precision')
Scores_SRP2['recall'] = classification(X_data=X_SRP2, cls_name=LogisticRegression(C=0.1), scoring_method='recall')
Scores_SRP2['f1'] = classification(X_data=X_SRP2, cls_name=LogisticRegression(C=0.1), scoring_method='f1')
Scores_SRP2['roc_auc'] = classification(X_data=X_SRP2, cls_name=LogisticRegression(C=0.1), scoring_method='roc_auc')

Scores_SRP2

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.1),8,0.805556,0.803754,0.772527,0.783597,0.87329


## n=9, LR(C=0.01)

In [439]:
SRP3 = random_projection.SparseRandomProjection(n_components=9,random_state=1)
X_SRP3 = SRP3.fit_transform(X_norm)
X_SRP3.shape

(303, 9)

In [440]:
Scores_SRP3 = pd.DataFrame(columns = ['Classification_Method','#Features', 'accuracy','precision','recall','f1','roc_auc'])
Scores_SRP3['#Features'] = [9]
Scores_SRP3['Classification_Method'] = 'LR (C=0.01)'
Scores_SRP3

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.01),9,NaN,NaN,NaN,NaN,NaN


In [442]:
Scores_SRP3['accuracy'] = classification(X_data=X_SRP3, cls_name=LogisticRegression(C=0.01), scoring_method='accuracy')
Scores_SRP3['precision'] = classification(X_data=X_SRP3, cls_name=LogisticRegression(C=0.01), scoring_method='precision')
Scores_SRP3['recall'] = classification(X_data=X_SRP3, cls_name=LogisticRegression(C=0.01), scoring_method='recall')
Scores_SRP3['f1'] = classification(X_data=X_SRP3, cls_name=LogisticRegression(C=0.01), scoring_method='f1')
Scores_SRP3['roc_auc'] = classification(X_data=X_SRP3, cls_name=LogisticRegression(C=0.01), scoring_method='roc_auc')

Scores_SRP3

,Classification_Method,#Features,accuracy,precision,recall,f1,roc_auc
0,LR (C=0.01),9,0.806487,0.83697,0.721795,0.771543,0.869578


In [464]:
SRP = random_projection.SparseRandomProjection(n_components=4,random_state=1)
X_SRP = SRP.fit_transform(X)
X_SRP.shape
# n=4, C=1

(303, 4)

In [470]:
classification(X_data=X_SRP, cls_name=LogisticRegression(C=100), scoring_method='accuracy')

0.8007168458781363